In [ ]:
import coco_dataset
import pathlib
import show_utils
import common
import copy
from torchvision.transforms import v2
import transforms as mytf
import torch
import model

In [ ]:
inp_size = 368
coco_dataset = coco_dataset.CocoKeypoints(
    root=str(pathlib.Path("../coco") / "images" / "val2017"),
    annFile=str(pathlib.Path("../coco") / "annotations" / "annotations" / "person_keypoints_val2017.json"),
    transform=None)
i = 0

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    device = "cuda:0"
    model = torch.nn.DataParallel(model.openpose(), device_ids=[0])
    model.load_state_dict(torch.load("ff0.pth"))
    model.eval()
print(device)

In [ ]:
i = 9

In [ ]:
inp, pafs, heatmaps, paf_locs, anns, id = coco_dataset[i]  # ,mask_out

show_utils.show_coco(anns[0], anns[1], coco_dataset.coco, draw_bbox=False)

pred_pafs, pred_heatmaps = model(inp.unsqueeze_(0).to(device))

print(f"sample: {i}")
print(anns[0].size)
i += 1

In [ ]:
import post
inp = v2.ToPILImage()(inp.squeeze_(0).cpu())
with torch.no_grad():
    humans = post.post_process(pred_heatmaps.squeeze_(0).cpu(), pred_pafs.squeeze_(0).cpu(), inp.size)
    print("detected humans:", len(humans))

if humans:
    formated = post.format(humans)
    show_utils.draw_skeleton(inp, formated, connectivity=common.connect_skeleton) 
else:
    print("NO DETECTIONS")

In [ ]:
humans

In [ ]:
with torch.no_grad():
    heatmaps = pred_heatmaps.squeeze_(0).cpu().numpy()
    pafs = pred_pafs.squeeze_(0).cpu().numpy()

# paf_locs = np.array(paf_locs, dtype=float)
# mask_out = mask_out.squeeze_(0).numpy()

In [ ]:
# #---------heatmaps---------
# print(f"Heatmaps.size = {heatmaps.shape}")
# show_utils.plot_grid(heatmaps, 2, 10, (17,4))

# #---------pafs---------
# print(f"Pafs.size = {pafs.shape}")
# show_utils.plot_grid(pafs[[x for x in range(len(pafs)) if x%2==0]], 1, 17, (17, 2))
# show_utils.plot_grid(pafs[[x for x in range(len(pafs)) if x%2==1]], 1, 17, (17, 2))

# # ---------blend---------
# show_utils.blend(heatmaps, inp, rows=7, cols=3, figsize=(15, 30))
# show_utils.blend(paf_locs, inp, rows=6, cols=3, figsize=(15, 30))
# # show_utils.blend([mask_out], inp, rows=1, cols=2, figsize=(12, 4))

# # ---------surface---------
# show_utils.surface(heatmaps[0])

# #---------quiver---------
# show_utils.pafs_quiver_combined(pafs, size=(46,46))
# show_utils.pafs_quiver(pafs, size=(46,46))